In [1]:
import pandas as pd
import os
import glob

In [2]:
# Working directory
project_dir = '/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml'
os.chdir(project_dir)

In [3]:
def get_prediction_file_paths(directory):
    # Use glob to get all csv files in the directory
    csv_files = glob.glob(os.path.join(directory, '*prediction*.csv'))
    return csv_files

In [4]:
def postprocess_predictions(prediction_df, prediction_col='pred', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [5]:
def maximum_return(prediction_df, prediction_col='ret', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [6]:
def create_result(prediction_parent_path, result_file_name=None, period='month', max_return=False):
    # Get the prediction data paths
    prediction_data_paths = get_prediction_file_paths(prediction_parent_path)
    
    # Postprocess the prediction and append all the results together
    results = pd.DataFrame()
    for df_path in prediction_data_paths:
        df = pd.read_csv(df_path)
        if max_return:
            if period=='month':
                year_vret = maximum_return(df, prediction_col='ret', period='month')
            elif period=='quarter':
                year_vret = maximum_return(df, prediction_col='retq', period='quarter')
            else:
                print('Please input period as either month or quarter')
        else:
            year_vret = postprocess_predictions(df, period=period)
        results = pd.concat([results, year_vret]).reset_index(drop=True)
    
    # Sort the results
    sorted_results = results.sort_values(by=['date', 'rank'],  ascending=[True, True]).reset_index(drop=True)
    
    # Save the sorted results to the same parent directory if file name is given
    if result_file_name:
        sorted_results.to_csv(f'{prediction_parent_path}/{result_file_name}', index=False)
        
    return sorted_results

In [7]:
def compute_returns(path):
    
    # Load data
    df = pd.read_csv(path)

    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

In [8]:
def compute_returns_df(df):
    
    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

### Current Quarterly New Restricted

In [9]:
period = 'quarter'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_restricted'
result_file_name = 'result.csv'

In [10]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [11]:
compute_returns_df(results)

/tmp/ipykernel_479786/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.016267
1,1.0,0.026286
2,2.0,0.029964
3,3.0,0.032864
4,4.0,0.036701
5,5.0,0.031517
6,6.0,0.038004
7,7.0,0.042957
8,8.0,0.043766
9,9.0,0.051383


### Current Monthly Restricted

In [12]:
period = 'month'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_restricted'
result_file_name = 'result.csv'

In [13]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [14]:
compute_returns_df(results)

/tmp/ipykernel_479786/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.002734
1,1.0,0.004993
2,2.0,0.007454
3,3.0,0.008746
4,4.0,0.009057
5,5.0,0.014019
6,6.0,0.013352
7,7.0,0.014316
8,8.0,0.017810
9,9.0,0.024079


### Monthly New Vars

In [15]:
compute_returns('kevin/output/prediction/monthly_new_vars/result.csv')

/tmp/ipykernel_479786/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.005073
1,1.0,0.008334
2,2.0,0.010095
3,3.0,0.010380
4,4.0,0.011332
5,5.0,0.013042
6,6.0,0.011259
7,7.0,0.011464
8,8.0,0.012775
9,9.0,0.014816


### Quarterly New Vars

In [16]:
compute_returns('kevin/output/prediction/quarterly_new_vars/result.csv')

/tmp/ipykernel_479786/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.028847
1,1.0,0.026470
2,2.0,0.031129
3,3.0,0.030091
4,4.0,0.031548
5,5.0,0.035604
6,6.0,0.034563
7,7.0,0.032160
8,8.0,0.032736
9,9.0,0.031680


### Monthly Old Vars

In [17]:
compute_returns('kevin/output/prediction/monthly_old_vars/result.csv')

/tmp/ipykernel_479786/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.003681
1,1.0,0.007112
2,2.0,0.009797
3,3.0,0.009424
4,4.0,0.011221
5,5.0,0.011056
6,6.0,0.011664
7,7.0,0.010712
8,8.0,0.012864
9,9.0,0.017309


### Analyzing result files

In [18]:
def analyze_result_files(result_dir):
    result = pd.read_csv(result_dir)
    result['year'] = result['date'].str[:4]
    summary = pd.pivot_table(data=result, index='year', columns='rank', values='port_ret', aggfunc='mean')

    return summary

In [19]:
monthly_restricted_result_dir = 'kevin/output/prediction/monthly_new_restricted/result.csv'
quarterly_restricted_result_dir = 'kevin/output/prediction/quarterly_new_restricted/result.csv'

In [20]:
analyze_result_files(monthly_restricted_result_dir)

rank,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
year,,,,,,,,,,
1985,0.007782,0.011813,0.019460,0.017572,0.021380,0.022893,0.023536,0.029384,0.026642,0.025835
1986,0.014892,0.009001,0.005955,0.012633,-0.014289,0.142857,NaN,NaN,NaN,NaN
1988,0.006600,0.011435,0.014111,0.013456,0.012173,0.013616,0.015319,0.013950,0.018627,0.020710
1989,0.019809,0.016106,0.019523,0.022818,0.026783,0.021589,0.023746,0.024094,0.023030,0.019958
1990,-0.035582,-0.007281,-0.000781,-0.000133,-0.011434,0.001176,-0.002714,0.000332,0.000488,0.000400
1991,0.031874,0.020255,0.026786,0.028302,0.024785,0.026323,0.029717,0.026980,0.021608,0.022503
1992,0.000234,0.007593,0.007893,0.009300,0.012536,0.009336,0.010672,0.015046,0.009858,0.019359
1993,0.001306,0.011367,0.008800,0.010960,0.004245,0.044392,-0.002360,0.011265,0.050894,NaN
1994,-0.009037,-0.009065,-0.011872,-0.005565,-0.004598,0.004803,0.003193,0.005398,0.002830,0.012204


In [21]:
analyze_result_files(quarterly_restricted_result_dir)

rank,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
year,,,,,,,,,,
1985,0.053303,0.081361,0.072383,0.075900,0.089916,0.076854,0.078322,0.075168,0.083178,0.070111
1986,0.042059,0.047332,0.018916,0.010148,0.019203,0.038678,0.052373,0.058832,0.042755,0.044340
1988,-0.017456,0.026807,0.046970,0.027694,0.048664,0.040783,0.044029,0.039901,0.030149,0.047634
1989,0.080804,0.058453,0.057417,0.041163,0.068550,0.063898,0.072048,0.079395,0.068635,0.073671
1990,-0.095201,-0.058611,-0.023418,-0.003278,-0.007505,0.009274,-0.006832,0.025546,0.078994,0.121227
1991,0.079262,0.082522,0.074682,0.087371,0.118864,0.072634,0.064356,0.085738,0.068568,0.076965
1992,0.004061,0.019968,0.040480,0.037356,0.027517,0.033554,-0.002619,0.001368,-0.033281,-0.069031
1993,0.050586,0.038850,-0.006097,0.016104,0.020981,0.021945,0.035432,0.044193,0.030676,0.035023
1995,0.070939,0.081300,0.068140,0.094021,0.091878,0.086546,0.090996,0.072398,0.076750,0.097220


### RF results reference

In [22]:
quarterly_restricted_result_dir_RF = 'Info Processing and Mutual Funds/new_input/results/result_w_c_q_new_all_restrict_95_stocks.csv'
quarterly_restricted_result_dir_RF = 'Info Processing and Mutual Funds/new_input/results/result_w_c_q_new_all_restrict_95.csv'

In [23]:
analyze_result_files(quarterly_restricted_result_dir_RF)

rank,0,1,2,3,4,5,6,7,8,9
year,,,,,,,,,,
1986,-0.042276,0.000715,0.029379,0.030065,0.037146,0.052625,0.051450,0.055685,0.061998,0.054150
1987,-0.047072,-0.034725,-0.020583,-0.014037,-0.004612,0.013922,0.007724,0.015038,0.038979,0.057305
1988,-0.032986,-0.013332,0.016615,0.027845,0.039641,0.027500,0.036896,0.029400,0.062737,0.069541
1989,-0.039194,0.001860,0.021272,0.041653,0.046331,0.039573,0.048253,0.069729,0.081896,0.093934
1990,-0.180627,-0.135499,-0.088966,-0.049471,-0.040899,-0.012314,-0.025692,-0.014494,-0.000844,0.008721
1991,0.084129,0.082954,0.050934,0.059230,0.049883,0.066490,0.068891,0.083475,0.116456,0.162577
1992,-0.041512,0.009157,0.014145,0.038007,0.030571,0.029338,0.023477,0.024085,0.005520,-0.010597
1993,-0.015621,0.022742,0.016251,0.047191,0.018877,0.038794,0.055268,0.062425,0.061826,0.051368
1994,-0.107817,-0.054129,-0.010377,0.004112,0.000562,0.006738,-0.000606,0.007396,-0.005335,0.043673


In [24]:
compute_returns(quarterly_restricted_result_dir_RF)

/tmp/ipykernel_479786/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.035549
1,1.0,-0.002851
2,2.0,0.012681
3,3.0,0.026528
4,4.0,0.030880
5,5.0,0.031569
6,6.0,0.039013
7,7.0,0.044643
8,8.0,0.047770
9,9.0,0.066568
